In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [3]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1747998297301, experiment_id='1', last_update_time=1747998297301, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [4]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    
    df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    
    #Standart deviation of the trips duration in January
    print('Standart deviation: ', df.duration.std())
    
    # Fraction of the records between 1 and 60 minutes
    print('Fraction: ', ((df.duration >= 1) & (df.duration <= 60)).mean())
    
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    return df

In [6]:
df_train = read_dataframe('data/yellow_tripdata_2023-01.parquet')
df_val = read_dataframe('data/yellow_tripdata_2023-02.parquet')

Standart deviation:  42.59435124195458
Fraction:  0.9812202822125979
Standart deviation:  42.84210176105113
Fraction:  0.9800944077722545


In [ ]:
df_jan = read_dataframe("https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet")
df_feb = read_dataframe("https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet")
df_march = read_dataframe("https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-03.parquet")

In [7]:
categorical = ['PULocationID','DOLocationID']
df_train[categorical] = df_train[categorical].astype(str)

categorical = ['PULocationID','DOLocationID']
df_val[categorical] = df_val[categorical].astype(str)

In [8]:
dv = DictVectorizer()

train_dicts = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

: 

In [ ]:
val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

print('Dimensionality: ', X_train.shape)

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred_train = lr.predict(X_train)
y_pred_val = lr.predict(X_val)

print('RMSE on train: ', mean_squared_error(y_train, y_pred_train, squared=False))
print('RMSE on val: ', mean_squared_error(y_val, y_pred_val, squared=False))

In [ ]:
sns.distplot(y_pred_train, label='prediction')
sns.distplot(y_train, label='actual')

plt.legend()

In [ ]:
sns.distplot(y_pred_val, label='prediction')
sns.distplot(y_val, label='actual')

plt.legend()

In [ ]:
with mlflow.start_run():
    mlflow.set_tag("ML eng", "Ulya")
    
    mlflow.log_param("train-data-path", 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
    mlflow.log_param("val-data-path", "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet")
    
    alpha = 0.01
    
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_param("rmse", rmse)